In [2]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [23]:
import pandas as pd
import numpy as np

test_seg_list = []
test_seg_list.append(0)

def div(a):
#     print("Running seg{0} ... ".format(a))
    df = pd.read_csv("./data/segData/df_seg%s.csv"%a)
    df.drop(["Unnamed: 0", "MemberId"], axis=1, inplace=True)
#     print("Sum of data(for this seg): {0} ".format(len(df)))
    msk = np.random.rand(len(df)) < 0.3
    train = df[msk]
    test = df[~msk]
    
#     print(train["SalesOrderSlaveTotalPayment"].max())
#     print(train["SalesOrderSlaveTotalPayment"].min())
    
    train["SalesOrderSlaveTotalPayment"] = norm(train["SalesOrderSlaveTotalPayment"])
#     print(train["SalesOrderSlaveTotalPayment"])
    test["SalesOrderSlaveTotalPayment"] = norm(test["SalesOrderSlaveTotalPayment"])
    
#     print("Sum of training data(for this seg): {0}".format(len(train)))
#     print("Sum of testing data(for this seg): {0}".format(len(test)))
    before = test_seg_list[-1]
    test_seg_list.append(len(test))
    
#     print("...")
    return train, test

def norm(s):
    result = (s - s.min())/(s.max()-s.min())
    return(result)

trainingData, testingData = div(2)
# df0, df1 = div(2)
# trainingData=pd.concat([trainingData,df0],ignore_index=True)  
# print(len(trainingData))
# testingData=pd.concat([testingData,df1],ignore_index=True)  
# print(len(testingData))

for i in range(3,21):
    df0, df1 = div(i)
    trainingData = pd.concat([trainingData,df0],ignore_index=True)  
#     print(len(trainingData))
    testingData = pd.concat([testingData,df1],ignore_index=True)  
#     print(len(testingData))

num = testingData["num"]

trainingData["num"] = norm(trainingData["num"])
testingData["num"] = norm(testingData["num"])
# print("Sum of training data: {0}".format(len(trainingData)))
# print("Sum of testing data: {0}".format(len(testingData)))
# print(test_seg_list)

/home/sopper08/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sopper08/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

def get_testing_labels(training_samples, training_labels, testing_samples):
    normalizer       = Normalizer().fit(training_samples)
    training_samples = normalizer.transform(training_samples)
    testing_samples  = normalizer.transform(testing_samples)

    pca              = PCA(n_components=0.8, whiten=True).fit(training_samples)
    training_samples = pca.transform(training_samples)
    testing_samples  = pca.transform(testing_samples)

    svm_classifier   = SVC(C=2.8).fit(training_samples, training_labels)
    return svm_classifier.predict(testing_samples)

def main():
    # Read training and testing data
    training_df  = trainingData
    testing_df   = testingData
    print("Training set has {0[0]} rows and {0[1]} columns".format(training_df.shape))
    print("Test set has {0[0]} rows and {0[1]} columns".format(testing_df.shape))

    # Get training samples and labels from training data
    training_labels     = training_df["label"].values
    training_samples    = training_df.drop(["label"], axis = 1).values
    # Get testing samples from testing data
    real_testing_label  = testing_df["label"]
    
    testing_samples     = testing_df.drop(["label"], axis = 1).values
    testing_labels      = get_testing_labels(training_samples, training_labels, testing_samples)

    # Writing testing labels to CSV
    df_label   = pd.DataFrame(testing_labels, columns=["label"])
    df_label.index     += 1
    df_label.index.name = "label"
    df_label["real_testing_label"] = real_testing_label
    df_label["num"] = num
    df_label.to_csv('./data/training&testingData/testing_labels_df.csv', sep=',', index = False)
    print('testingLabel finish!')
    
    # 計算準確率
    df_label.dropna(inplace=True)
    df_label_int = df_label.astype("int")
    df_label_int['diff'] = df_label_int['real_testing_label']-df_label_int['label']
    len_df = len(df_label_int)
    accuracy = (len_df - df_label_int['diff'].abs().sum())/len_df*100
    print("all Accuracy: {0}%".format(accuracy))
    
    
    df_label_int["both1"] = 0
    df_label_int["l1r0"] = 0
    df_label_int["l0r1"] = 0
    df_label_int["both0"] = 0

    df_label_int["both1"][(df_label_int["label"]==True)&(df_label_int["real_testing_label"]==True)]=1
    df_label_int["l1r0"][(df_label_int["label"]==True)&(df_label_int["real_testing_label"]==False)]=1
    df_label_int["l0r1"][(df_label_int["label"]==False)&(df_label_int["real_testing_label"]==True)]=1
    df_label_int["both0"][(df_label_int["label"]==False)&(df_label_int["real_testing_label"]==False)]=1
    print(df_label_int.groupby(by="num")["both1", "l1r0", "lor1", "both0"].sum())
    

if __name__ == "__main__":
    main()

Training set has 225467 rows and 40 columns
Test set has 526983 rows and 40 columns
testingLabel finish!
all Accuracy: 65.85689833808365%
      both1   l1r0  lor1  both0
num                            
2    100236  76293  1878   1429
3     64111  36682   871    544
4     43427  20641   500    251
5     30939  12538   313    115
6     22759   8329   200     72
7     17078   5593   138     35
8     13148   3992    81     18
9     10248   2905    65     13
10     8259   2099    41     13
11     6748   1563    29      4
12     5461   1196    22      4
13     4578    981    15      5
14     3780    731     5      0
15     3190    599     0      0
16     2753    455     3      0
17     2378    388     0      0
18     2042    311     0      1
19     1805    251     1      0
20     1610    218     1      0
